In [35]:
from patsy import dmatrices
import pandas as pd
from sklearn.linear_model import LogisticRegression
import statsmodels.discrete.discrete_model as sm


# read in the data & create matrices
dat = pd.read_csv("data/TrainExer5-5-2.txt",\
                  sep = '\t')

dat.columns = ['id','response','male','activite','age']

dat.head()

,id,response,male,activite,age
0,2,1,1,0,50
1,3,1,1,0,40
2,4,1,1,0,36
3,5,1,1,0,28
4,6,1,1,0,70


In [44]:
dummy_response = pd.get_dummies(dat['response'], prefix='response')
print dummy_response.head()

   response_0  response_1
0         0.0         1.0
1         0.0         1.0
2         0.0         1.0
3         0.0         1.0
4         0.0         1.0


In [60]:
# TODO: coeffients from sklearn.linear_model and statsmodel are different!!!
# sklearn output
model = LogisticRegression(fit_intercept = True, C = 1e9)
dat['age2'] = (dat['age']*0.1) ** 2
X = dat[['male', 'activite','age','age2']]
y = dat['response']
mdl = model.fit(X, y)
print model.coef_


# sm
X['intercept'] = 1.0 # manually add intercept
logit = sm.Logit(y, X)
sm_model = logit.fit()
sm_model.summary()

[[ 0.96679321  0.91755598  0.06843405 -0.06736053]]
Optimization terminated successfully.
         Current function value: 0.650135
         Iterations 5


/Users/CJ/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               response   No. Observations:                  924
Model:                          Logit   Df Residuals:                      919
Method:                           MLE   Df Model:                            4
Date:                Fri, 16 Sep 2016   Pseudo R-squ.:                 0.06190
Time:                        10:46:07   Log-Likelihood:                -600.73
converged:                       True   LL-Null:                       -640.36
                                        LLR p-value:                 2.482e-16
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
male           0.9668      0.159      6.093      0.000         0.656     1.278
activite       0.9176      0.185      4.963      0.000         0.555     1.280
age            0.0684      0.036      1.921      0.055        -0.001     0.138
age2          -0.0674      0.034     -1.975      0.048        -0.134    -0.001
intercept     -2.4619      0.890     -2.766      0.006        -4.207    -0.717
==============================================================================
"""

(c)Test the null hypothesis H0: β1 = β2 = 0 versus H1: no restrictions on β1 and β2, using a likelihood ratio test.

**ans:**: 

Testing for a set of parameter restrictions
We want to compare
logit model without parameter restrictions and estimates b1
logit model with m parameter restrictions and estimates b0
The null hypothesis is that the m parameter restrictions are correct.
To compute the test statistic we need
L(b1): maximum likelihood value in full model
L(b0): maximum likelihood value in restricted model
Test statistic:
$$ LR = −2(log(L(b_0)) − log(L(b_1))) ≈ χ^2(m)$$ 
where m is the number of restrictions.

In [51]:
# fit the restricted model without b1 and b2
X = dat[['age','age2']]
y = dat['response']

X['intercept'] = 1.0 # manually add intercept
logit = sm.Logit(y, X)
model_restrict = logit.fit()
model_restrict.summary()

Optimization terminated successfully.
         Current function value: 0.687749
         Iterations 4


/Users/CJ/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               response   No. Observations:                  924
Model:                          Logit   Df Residuals:                      921
Method:                           MLE   Df Model:                            2
Date:                Fri, 16 Sep 2016   Pseudo R-squ.:                0.007624
Time:                        10:43:06   Log-Likelihood:                -635.48
converged:                       True   LL-Null:                       -640.36
                                        LLR p-value:                  0.007582
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
age            0.1011      0.034      2.947      0.003         0.034     0.168
age2          -0.0992      0.033     -3.021      0.003        -0.164    -0.035
intercept     -2.3686      0.860     -2.755      0.006        -4.054    -0.684
==============================================================================
"""

In [62]:
full = sm_model.llf
restricted = model_restrict.llf
LR = -2*(restricted - full)
LR

69.509659248478783

In [65]:
import scipy.stats as stats
full_model = sm_model
restricted_model = model_restrict
df = full_model.df_model - restricted_model.df_model
crit = stats.chi2.ppf(q = 0.95, # Find the critical value for 95% confidence*
                      df = df)   # Df = number of variable categories - 1

p_value = 1 - stats.chi2.cdf(x=LR,  # Find the p-value
                             df=df)
p_value

7.7715611723760958e-16

In [64]:
crit

5.9914645471079799